# python:类 学习笔记

## 1. 类介绍

### 1.1 类对象

类对象支持两种操作: **属性引用**和**实例化**.

有效的属性名称是类对象被创建时存在于类命名空间中的**所有名称**, 例如:

In [2]:
class MyClass:
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'

In [3]:
MyClass.i

12345

In [33]:
MyClass.f # 这是一个函数对象

<function __main__.MyClass.f(self)>

类属性也可以被赋值，因此可以通过赋值来更改 MyClass.i 的值

In [8]:
MyClass.i = 123
MyClass.i

123

In [11]:
# __doc__ 也是一个有效的属性，将返回所属类的文档字符串
MyClass.__doc__

'A simple example class'

实例化: 可以把类对象视为是返回该类的一个新实例的不带参数的函数, 例如:

In [29]:
x = MyClass()
x.i, x.f()

(12345, 'hello world')

给类定义一个 `__init__()` 方法, 在类被实例化时就该方法会自动被调用(初始化):

In [51]:
class MyClass:
    """A simple example class"""
    i = 12345

    def __init__(self):
        self.data = []

    def f(self):
        return 'hello world'
    
    def h(self, x, y):
        print(x+y)

In [35]:
x = MyClass() # x是一个实例对象
x.data

[]

`__init__()` 方法还可以有额外参数以实现更高灵活性。 在这种情况下，提供给**类实例化的参数**将被传递给 `__init__()`。 例如，:

In [36]:
class Complex:
    def __init__(self, realpart, imagpart):
        self.r = realpart
        self.i = imagpart

In [37]:
x = Complex(3.0, -4.5) # 提供给类实例化的参数: 3.0, -4/5
x.r, x.i

(3.0, -4.5)

### 1.2 实例对象

实例对象理解的唯一操作是属性引用. 属性分两种: 数据属性和方法属性.
- 数据属性不需要在类定义中声明, 它们在被赋值时产生和改变
- 方法属性是从数于对象的函数. 一个类中定义的所有函数对象都是其实例的方法属性.

In [52]:
# 数据属性:
x = MyClass()
x.counter = 1
while x.counter < 10:
    x.counter = x.counter * 2
print(x.counter)
del x.counter

16


In [43]:
# 方法属性, 注意 x.f 是方法对象, 是一个方法; MyClass.f 是函数对象, 是一个函数
type(x.f), type(MyClass.f)

(method, function)

In [65]:
x.h(1,2) # 方法对象被调用时需要提供定义中除self外的其它参数.

3


In [66]:
MyClass.h(x, 1 ,2) # 函数对象被调用时需要提供定义中的所有参数, 相当于x.h()

3


### 1.3 方法对象

如前面的例子所示, 方法的特殊之处就在于**实例对象会作为函数的第一个参数被传入**, 方法的第一个参数常常被命名为 `self`。在示例中，调用 x.f() 其实就相当于 MyClass.f(x)。

### 1.4 类变量和实例变量

一般来说，实例变量用于每个实例的唯一数据，而类变量用于类的所有实例共享的属性和方法:

```python
class Dog:

    kind = 'canine'         # class variable shared by all instances

    def __init__(self, name):
        self.name = name    # instance variable unique to each instance

>>> d = Dog('Fido')
>>> e = Dog('Buddy')
>>> d.kind                  # shared by all dogs
'canine'
>>> e.kind                  # shared by all dogs
'canine'
>>> d.name                  # unique to d
'Fido'
>>> e.name                  # unique to e
'Buddy'
```

注意, 不要把 mutable对象(如列表和字典) 设置为类变量, 因为这样容易把实例的特性混入类的特性.

```python
class Dog:

    tricks = []             # mistaken use of a class variable

    def __init__(self, name):
        self.name = name

    def add_trick(self, trick):
        self.tricks.append(trick)

>>> d = Dog('Fido')
>>> e = Dog('Buddy')
>>> d.add_trick('roll over')
>>> e.add_trick('play dead')
>>> d.tricks                # unexpectedly shared by all dogs
['roll over', 'play dead']
```

```python
class Dog:

    def __init__(self, name):
        self.name = name
        self.tricks = []    # creates a new empty list for each dog

    def add_trick(self, trick):
        self.tricks.append(trick)

>>> d = Dog('Fido')
>>> e = Dog('Buddy')
>>> d.add_trick('roll over')
>>> e.add_trick('play dead')
>>> d.tricks
['roll over']
>>> e.tricks
['play dead']
```

### 1.5 补充说明

方法可以通过使用`self` 参数的方法属性调用其他方法:

In [1]:
class Bag:
    def __init__(self):
        self.data = []

    def add(self, x):
        self.data.append(x)

    def addtwice(self, x):
        self.add(x)
        self.add(x)

In [ ]:
b = Bag()
b.addtwice(1)
b.data

[1, 1]

每个值都是一个对象，因此具有 类 （也称为 类型），并存储为 `object.__class__`:

In [8]:
's'.__class__

str

## 2. 继承

派生类定义的语法如下所示:

```python
class DerivedClassName(BaseClassName):
    <statement-1>
    .
    .
    .
    <statement-N
```

当基类定义在另一个模块中的时候:

```python
class DerivedClassName(modname.BaseClassName):
```

方法引用将按以下方式解析：搜索相应的类属性，如有必要将按基类继承链逐步向下查找，如果产生了一个函数对象则方法引用就生效。

Python有两个内置函数可被用于继承机制：

- 使用 `isinstance()` 来检查一个实例的类型: `isinstance(obj, int)` 仅会在 `obj.__class__` 为 `int` 或某个派生自 `int` 的类时为 `True`。

- 使用 `issubclass()` 来检查类的继承关系: `issubclass(bool, int)` 为 `True`，因为 `bool` 是 `int` 的子类。 但是，`issubclass(float, int)` 为 `False`，因为 `float` 不是 `int` 的子类。


### 2.1 多重继承

带有多个基类的类定义语句如下所示:

```python
class DerivedClassName(Base1, Base2, Base3):
    <statement-1>
    .
    .
    .
    <statement-N>
```

如果某一属性在 DerivedClassName 中未找到，则会到 Base1 中搜索它，然后（递归地）到 Base1 的基类中搜索，如果在那里未找到，再到 Base2 中搜索，依此类推。